In [20]:
import os
import datetime
import dateutil
import tarfile
import requests
import pandas as pd
import pandas.io.sql as sqlio

%matplotlib inline

In [21]:
stream = requests.get('https://www.lrde.epita.fr/~ricou/pybd/projet/boursorama.tar', stream=True)

In [22]:
with open('boursorama.tar', 'wb') as f:
    for chunk in stream.iter_content(chunk_size=8192):
        if chunk:
            f.write(chunk)

In [23]:
with tarfile.open('boursorama.tar', 'r') as tar:
    tar.extractall()

In [24]:
os.listdir('boursorama/2020')[:3]

['compB 2020-04-08 17:22:02.263233',
 'compB 2020-01-13 11:52:01.507632',
 'compA 2020-08-07 16:51:02.060904']

In [109]:
# Please select the year that you would like to study :
chosen_year = 2020

In [25]:
# Everything is in the .bz2 format, we need to decompress the file
import bz2

dir_path = os.path.join('boursorama', chosen_year)

for bz2_file in os.listdir(dir_path):
    if not bz2_file.endswith('.bz2'):
      continue

    file_name = bz2_file[:-4]
    with bz2.open(dir_path + '/' + bz2_file, 'rb') as bz2_format, open(dir_path + '/' + file_name, 'wb') as file_format:
      file_format.write(bz2_format.read())

    os.remove(dir_path + '/'+ bz2_file)

In [100]:
import glob
from dateutil.parser import parse

file_paths_compB = glob.glob('boursorama/2020/compB*')
fp_dict_compB = {parse(x.split('compB ')[1]):pd.read_pickle(x) for x in file_paths_compB}
compB = pd.concat(fp_dict_compB)

## Cleaning the dataFrame

In [ ]:
### Here is a resume of every cleaning process that we did on the given data (to present at the soutenance)  ###

# 1. Verifying and dropping the None and NaN value of our dataFrames
# 2. Verifying if some companies did not change their names in 5 years and edit the name like LNC which became BASSAC
# 3. Verifying if the symbol column and symbol header are exaclty the same
# 4. Checking if the 'last' format is correct (positive float) && Changing the value that does not match the format such as the ones that ends with a '(c)' or a '(s)'
    # End with (c) : There is nothing to convert or anything, it just to signal that the value can be `convertible` or is a `cum_dividend`
    # End with (s) : There is nothing to convert or anything, it just to signal that the value is a `subscription right/subscription warrant`
# 5. Checking if the 'volume' format is correct (positive integer) && Dropping every volume value that are negative such as the value from the XXX company or the XXX company
# 6. Checking if the 'name' format is correct (all caps) && Edit the names that are not like 'PLASTiVALOIRE' into 'PLASTIVALOIRE' (only one found)
# 7. Checking if the 'date' format is correct : YYYY-MM-DD HH:MM:SS.microsecondes with YYYY = name of the dir
# 8. Checking if the 'symbol' format is correct : 1rP* (only found FF11_VRAP that does not match but turns out it is not an error)
# 9. Drop the duplicates if there is any
# 10. Remove the day_off ? Verify is that is a good idea


# Note:
# After a discussion with an assistant, we don't really have to verify the outliers because big variation of values is something that is normal in the bourse area
# Some company has several symbol (ex : ASSYSTEM :[1rPASYT, 1rPASY]), we don't need to clean that it is a normal behavior

In [60]:
compB.head()

last  volume    symbol           name
                           symbol                                           
2020-04-08 17:22:02.263233 1rPAB      7.150  107638     1rPAB     AB SCIENCE
                           1rPABCA    6.120   18698   1rPABCA  ABC ARBITRAGE
                           1rPABEO    9.000    1723   1rPABEO           ABEO
                           1rPAKOM   14.000     120   1rPAKOM        AERKOMM
                           1rPAESCH   0.550       0  1rPAESCH   AES CHEMUNEX

In [107]:
# Step 1: Clean any Null or Nan value

has_null_value = compB.isnull().sum().sum()
has_nan_value = compB.isna().sum().sum()
if has_null_value + has_nan_value > 0:
    print('The DataFrame contains any Null or Nan value, you need to clean it')
    # Clean the Null or Nan value

In [108]:
# Step 2: Check if a company has changed his name

compB.rename(columns={'symbol': 'symbol_column'}, inplace=True)

def symbol_linked_to_several_names(all_names):
    return len(all_names) > 1

symbol_and_names = compB.groupby('symbol_column')['name'].unique()
edited_names_list = list(filter(symbol_linked_to_several_names, symbol_and_names))
for last_name, new_name in edited_names_list:
  compB.loc[compB['name'] == last_name, 'name'] = new_name


In [87]:
# Step 3: Checking if the symbol of index and symbol column are the strictly the same and drop the column if that is the case
index_symbols = compB.index.get_level_values('symbol').unique()
column_symbols = compB['symbol_column'].unique()

if set(index_symbols) == set(column_symbols):
    compB.drop(columns='symbol_column',inplace=True)

compB.head()

last  volume           name
                           symbol                                 
2020-04-08 17:22:02.263233 1rPAB      7.150  107638     AB SCIENCE
                           1rPABCA    6.120   18698  ABC ARBITRAGE
                           1rPABEO    9.000    1723           ABEO
                           1rPAKOM   14.000     120        AERKOMM
                           1rPAESCH   0.550       0   AES CHEMUNEX

In [110]:
# Step 4 : Transform the dataFrame to make it more readable (according to me) -> Not very optimized, this step might be removed
def concat_tuple_as_string(t):
    return ' '.join(map(str, t))

index_as_strings = compB.index.map(concat_tuple_as_string)
compB['timestamped_symbol'] = index_as_strings
compB.reset_index(drop=True, inplace=True)
compB.head()

,last,volume,symbol_column,name,timestamped_symbol
0,7.15,107638,1rPAB,AB SCIENCE,2020-04-08 17:22:02.263233 1rPAB
1,6.12,18698,1rPABCA,ABC ARBITRAGE,2020-04-08 17:22:02.263233 1rPABCA
2,9.00,1723,1rPABEO,ABEO,2020-04-08 17:22:02.263233 1rPABEO
3,14.00,120,1rPAKOM,AERKOMM,2020-04-08 17:22:02.263233 1rPAKOM
4,0.55,0,1rPAESCH,AES CHEMUNEX,2020-04-08 17:22:02.263233 1rPAESCH


In [114]:
# Step 5: Check if each columns match the expected value && changes the column to match the format
# Last: Has to be a positive float
# Volume: Has to be a natural positive number
# Name : Has to be written in case
# timpestamped_symbol : Has to have the following format -> 2020-MM-DD HH:MM:SS.microsecondes 1rP*


# Last : Convert the last column to the expected format (remove (c) and (s)) , Picked that code to Ilays
def format_last(x):
  try:
    return float(x)
  except:
    return float(x.split('(')[0].replace(' ', ''))

compB['last'] = compB['last'].apply(format_last)
compB = compB[compB['last'] >= 0]

# Volume : Remove negatif value
compB = compB[compB['volume'] >= 0]

# Name
compB['name'] = compB['name'].str.upper()

# Timestamped_symbol

pattern = fr'{chosen_year}-\d{{2}}-\d{{2}} \d{{2}}:\d{{2}}:\d{{2}}\.\d{{6}} (1rP.*|FF11_VRAP)'
valid_timestamped_symbol = compB['timestamped_symbol'].str.match(pattern)
if not valid_timestamped_symbol.all():
  invalid_timestamped_symbols = compB[~valid_timestamped_symbol]['timestamped_symbol']
  print('The timpestamped_symbol column is not valid', invalid_timestamped_symbols)

In [ ]:
# Step 6: Check and drop the duplicate

duplicates = compB.duplicated(subset=['timestamped_symbol'], keep=False)
if duplicates.any():
  print('There are some duplicated on this df, we need to do something')